In [ ]:
!pip install stop_words

In [ ]:
!pip install pymorphy2

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import time
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path_to_file = '/content/drive/MyDrive/Colab Notebooks/data/Obshestvo_Potrebleniya.txt'
path_to_file_filtered = '/content/drive/MyDrive/Colab Notebooks/data/Obshestvo_Potrebleniya_filtered.txt'
#path_to_file = '/content/drive/MyDrive/Colab Notebooks/data/evgenyi_onegin.txt'

In [ ]:
with open(path_to_file) as f:
  lines = f.readlines()
  filtered = list(filter(lambda x: x !='\n', lines))

with open(path_to_file_filtered, "w") as f:
  f.writelines(filtered)

In [ ]:
text = open(path_to_file_filtered, 'rb').read().decode(encoding='utf-8')

print('Length of text: {} characters'.format(len(text)))

Length of text: 543852 characters


In [ ]:
print(text[:500])

Общество потребления
Жан Бодрийяр
Философия – Neoclassic
Жан Бодрийяр (1929–2007) – изобретатель терминов «гиперреальность» и «симулякр», «великий провокатор» современной философии, автор произведений, совершенно изменивших само отношение к жизни современного общества, один из самых значимых мыслителей эпохи постмодернизма, которого, собственно, многие исследователи считают не иначе как «отцом-создателем» постмодернистской философии. Политика и экономика, культура и масс-медиа, искусство и даже 


In [ ]:
text = text + text

In [ ]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

152 unique characters


In [ ]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [ ]:
text_as_int, text, len(text_as_int), len(text)

(array([ 98, 114, 138, ..., 121,   9,   0]),
 "Общество потребления\nЖан Бодрийяр\nФилософия – Neoclassic\nЖан Бодрийяр (1929–2007)\xa0– изобретатель терминов «гиперреальность» и «симулякр», «великий провокатор» современной философии, автор произведений, совершенно изменивших само отношение к жизни современного общества, один из самых значимых мыслителей эпохи постмодернизма, которого, собственно, многие исследователи считают не иначе как «отцом-создателем» постмодернистской философии. Политика и экономика, культура и масс-медиа, искусство и даже мода – вот лишь немногое, на что, вольно или невольно, удалось радикально повлиять Бодрийяру.\nОбщество потребления – общество самообмана. Общество, которое гонится за бессмысленной и бессознательной иллюзией счастья. Общество, в котором сама необходимость потребления давно уже стала иррациональной самоцелью, «индустрия красоты» приравняла женское тело к «товару», а «философия успеха» обесценивает человеческую индивидуальность.\nРабота Жана Бо

### train and target

In [ ]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

О
б
щ
е
с


In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Общество потребления\nЖан Бодрийяр\nФилософия – Neoclassic\nЖан Бодрийяр (1929–2007)\xa0– изобретатель терм'
'инов «гиперреальность» и «симулякр», «великий провокатор» современной философии, автор произведений, '
'совершенно изменивших само отношение к жизни современного общества, один из самых значимых мыслителей'
' эпохи постмодернизма, которого, собственно, многие исследователи считают не иначе как «отцом-создате'
'лем» постмодернистской философии. Политика и экономика, культура и масс-медиа, искусство и даже мода '


In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Общество потребления\nЖан Бодрийяр\nФилософия – Neoclassic\nЖан Бодрийяр (1929–2007)\xa0– изобретатель тер'
Target data: 'бщество потребления\nЖан Бодрийяр\nФилософия – Neoclassic\nЖан Бодрийяр (1929–2007)\xa0– изобретатель терм'


In [ ]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  
    
    inputs = tf.keras.layers.Input(batch_input_shape=[batch_size, None])

    x =     tf.keras.layers.Embedding(vocab_size, embedding_dim)(inputs)
    print(x.shape)
    x1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')(x)
    x = tf.keras.layers.concatenate([x,x1], axis=-1)
    
    print(x.shape)
    x2 = tf.keras.layers.GRU(rnn_units+embedding_dim,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')(x)
    x = tf.keras.layers.add([x,x2])
    
    print(x.shape)
    x3 = tf.keras.layers.GRU(rnn_units+embedding_dim,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform')(x)   
    
    x = tf.keras.layers.add([x,x3])   
    x = tf.keras.layers.Dense(vocab_size)(x)
    print(x.shape)

    model =tf.keras. Model(inputs=inputs, outputs=x)
    
        
    return model

In [ ]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

(64, None, 256)
(64, None, 1280)
(64, None, 1280)
(64, None, 152)


In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 152) # (batch_size, sequence_length, vocab_size)


In [ ]:
model.summary()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_12 (InputLayer)           [(64, None)]         0                                            
__________________________________________________________________________________________________
embedding_13 (Embedding)        (64, None, 256)      38912       input_12[0][0]                   
__________________________________________________________________________________________________
gru_39 (GRU)                    (64, None, 1024)     3938304     embedding_13[0][0]               
__________________________________________________________________________________________________
concatenate_11 (Concatenate)    (64, None, 1280)     0           embedding_13[0][0]               
                                                                 gru_39[0][0]              

In [ ]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 152), dtype=float32, numpy=
array([[ 2.01545246e-02,  1.83544494e-02, -8.57363362e-03, ...,
        -1.23351980e-02,  2.34190449e-02,  1.42449781e-03],
       [-8.99396464e-03, -2.82627419e-02, -1.90494489e-02, ...,
         1.37119079e-02,  8.54283292e-03,  9.82632488e-03],
       [-1.81280002e-02, -1.99523550e-02, -2.14487035e-02, ...,
        -1.11041926e-02, -5.89879639e-02,  2.76687741e-02],
       ...,
       [ 9.10355942e-04,  7.41243502e-03,  2.10567769e-02, ...,
         2.89781522e-02, -6.37050765e-03,  1.47726899e-02],
       [-4.94295591e-03,  9.17697046e-03,  8.62337463e-03, ...,
         1.89315416e-02, -1.57563090e-02,  1.46070020e-02],
       [-2.53134333e-02,  1.27909435e-02, -1.15423026e-02, ...,
         1.53278336e-02,  1.90059971e-02,  2.88787887e-05]], dtype=float32)>

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [ ]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print("-------------------------------------")
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 ': предоставленный самому себе, находясь в состоянии «творческой свободы», человек на досуге отчаянно'
-------------------------------------
Next Char Predictions: 
 "ydcuK-(kLW!ьнт0ВеTА:7p«аЯ—/эу3дДЛПZУ-фAмE©aqRcАеdО©№ ЮхЙ«[lLЦ(1ёOГкHптОы'Уpз= фёR\xa0IЗHHТдЧДiu0Е’’шЭК4"


### Train the model

In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 152)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       5.020194


In [ ]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [ ]:
!rm -rf "/content/drive/MyDrive/Colab Notebooks/training_checkpoints"

In [ ]:
!ls "/content/drive/MyDrive/Colab Notebooks/training_checkpoints"

ls: cannot access '/content/drive/MyDrive/Colab Notebooks/training_checkpoints': No such file or directory


In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = '/content/drive/MyDrive/Colab Notebooks/training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

### Execute the training

In [ ]:
EPOCHS = 25

In [ ]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/25
168/168 [==============================] - 102s 576ms/step - loss: 2.4446
Epoch 2/25
168/168 [==============================] - 100s 589ms/step - loss: 1.5791
Epoch 3/25
168/168 [==============================] - 99s 582ms/step - loss: 1.2663
Epoch 4/25
168/168 [==============================] - 100s 588ms/step - loss: 1.0659
Epoch 5/25
168/168 [==============================] - 100s 588ms/step - loss: 0.8784
Epoch 6/25
168/168 [==============================] - 99s 579ms/step - loss: 0.7028
Epoch 7/25
168/168 [==============================] - 100s 586ms/step - loss: 0.5610
Epoch 8/25
168/168 [==============================] - 101s 590ms/step - loss: 0.4630
Epoch 9/25
168/168 [==============================] - 99s 582ms/step - loss: 0.4015
Epoch 10/25
168/168 [==============================] - 100s 589ms/step - loss: 0.3626
Epoch 11/25
168/168 [==============================] - 100s 588ms/step - loss: 0.3381
Epoch 12/25
168/168 [==============================] - 99s 579ms/s

In [ ]:
model1 = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model1.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model1.build(tf.TensorShape([1, None]))

(1, None, 256)
(1, None, 1280)
(1, None, 1280)
(1, None, 152)


In [ ]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.5

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
text_ = generate_text(model1, start_string=u"именно поскольку она ")
print(text_)

именно поскольку она посвящена героям потребления. По крайней мере на Западе экзальтированных в бюрократическом обществе в силу технического и социального разделения труда и своей частичной функцией, они стремятся «заново сформировавшейся идеи о том, что у нас есть общество изобилия, обозначающих друг друга и отсылающих один к другому. В крайнем случае, они его обдуманно конструируют. Значит, нужно употребить в отношении результатов их деятельности, если хотят судить об этом объективно, «фея схва, как объект являетс


In [ ]:
text_ = generate_text(model1, start_string=u"реализует стремление к ")
print(text_)

реализует стремление к высшему классу и новое утверждение собственной классовой позиции). Точнее, чтение выступает как знак присоединения. К чему? К абстрактной общности, к виртуальному коллективу возможности уравниваются (уравнивание доходов, социальное перераспределение, одна и та же мода для всех, одни и те же программы на телевидении, всей объективированное сторону морали, который следует рассматривать. Раз это сказано, поп-арт оказывается одновременно полным местам, реальным группам. Она предстает как эластичност


In [ ]:
text_ = generate_text(model1, start_string=u"Котики и ")
print(text_)

Котики и репрессии в качестве подлинного принципа потребления.
68
Вот почему гигиеническая мания (страдающая фобией, навязчивая) всегда близка нам. В целом, однако, гигиеническая озабоченность порождает не патетическую мораль, а игровую: она «обходит» глубокие фантазмы в целях поверхностной, близости со всем тем организационным началом, которое включает фестиваль. В рамках этого фестиваля все спасения прогресса, напряжения, допинг, который следует рассматривать. Раз это сказано, поп-арт оказывается однов


In [ ]:
text_ = generate_text(model1, start_string=u"Я сижу на стуле и делаю домашнюю работу ")
print(text_)

Я сижу на стуле и делаю домашнюю работу в ваше собственное тело – интересно – хотя это та же самая схема «гармонического соединения функции и формы».
Поэтому «слаборазвитые» народы воспринималось как личная, «персонализируясь». В китчу в общественном плане, формальным равновесие от принципа наслаждения и гедонистической рентабельности. На самом деле только фантом утраченной искренности на сферу товара и таковым является эквивалентом и продолжением в XX в. той большой дрессировки, которая на протяжении всей области потребления, станови


In [ ]:
text_ = generate_text(model1, start_string=u"путешествие за хлебом ")
print(text_)

путешествие за хлебом роста. Принцип же роста оказывается безупречным; он гомогенен и стремится гомогенизировать все общество.
Основной вопрос, который возникает при этом, касается «бедности». Для идеалистов изобилия она является «объективной» реальности подлинности и видимости. Есть только излучение и получение знаков, и не случайна: «красочное» насилие и умиротворение повседневной жизни за то система считается неэффективной, другой. То, что могло быть новой веровой действенности, чем имеет торговой действенности, ч
